In [4]:
import re
import glob
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs

In [55]:
def get_player_id(year):
    '''
    Get the player ids from basketball reference for 
    the totals table in a given year
    @param int year: the year for a player
    @rtype list: list of player ids 
    '''
    url = 'http://www.basketball-reference.com/leagues/NBA_{}_totals.html'.format(year)
    website = requests.get(url).text
    soup = bs(website, 'lxml')
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')[1:]
    ids = [i.find_all('td') for i in rows]
    id_list = [i[0]['data-append-csv'] for i in ids if len(i)>0]
    return id_list

In [16]:
path = os.getcwd() + "\Season Data 2"

files = glob.glob(path + "/*.csv")

In [64]:
for file in files:
    data = pd.read_csv(file)
    year = file.split('_')[-1].split('.')[0]
    data["Year"] = year
    primary_list = get_player_id(year)
    data["ID"] = primary_list
    data = data[data["G"] != 0]
    data = data[["Player", "Tm", "G", "Year", "ID"]]
    data = data[data["Tm"] != "TOT"]
    data = data.sort_values(by = ["G"], ascending = False).drop_duplicates(["Player"], keep = "first")
    data["Player"] = data["Player"].apply(lambda x: "".join(x.split("*")[0]))
    data.to_csv(file)

In [65]:
dataframes = []

for file in files:
    dataframes.append(pd.read_csv(file))

In [66]:
mega_frame = pd.concat(dataframes)
mega_frame.to_csv("Full-table.csv")